# Trabalho 1 

### Bruno Baldissera 10724351
### Bruno Gazoni 7585037

### Primeiro, vamos importar as bibliotecas necessárias.

In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np

### Inicializando janela

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(500, 500, "Exercício Prático #01", None, None)
glfw.make_context_current(window)

### Capturando eventos de teclado e mouse

In [3]:
def key_event(window,key,scancode,action,mods):
    print('[key event] key=',key)
    print('[key event] scancode=',scancode)
    print('[key event] action=',action)
    print('[key event] mods=',mods)
    print('-------')
    
glfw.set_key_callback(window,key_event)

def mouse_event(window,button,action,mods):
    print('[mouse event] button=',button)
    print('[mouse event] action=',action)
    print('[mouse event] mods=',mods)
    print('-------')
glfw.set_mouse_button_callback(window,mouse_event)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec2.
* Definindo uma variável chamada mat_transformation do tipo mat4 (matriz 4x4).
* Usamos vec2, pois nosso programa (na CPU) irá enviar apenas duas coordenadas para plotar um ponto. Podemos mandar três coordenadas (vec3) e até mesmo quatro coordenadas (vec4).
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base em uma matriz de transformação, conforme estudado na Aula05.

In [4]:
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transf;
        void main(){
            gl_Position = mat_transf * vec4(position,0.0,1.0);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

In [5]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [6]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [7]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [8]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [9]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [10]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [11]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


In [12]:
# Dado o centro (c_x, c_y) da engrenagem, o raio r do círculo, a largura l, a altura h do retângulo e o número n de dentes, a função retorna
#  um vetor com as coordenadas dos n retângulos, fazendo rotações consecutivas em um dente.
def calcula_retangulos(c_x, c_y, r, l, h, n):
    r0 = np.zeros((n*4, 3))
    r0[0] = [c_x -(l/2), c_y + r + h, 1]
    r0[1] = [c_x + (l/2), c_y + r + h, 1]
    r0[2] = [c_x - (l/2), c_y + r, 1]
    r0[3] = [c_x + (l/2), c_y + r, 1]

    theta = (2*math.pi)/n
    
    T1 = np.array([[1, 0, c_x],
                  [0, 1, c_y],
                  [0, 0, 1]])
    R = np.array([[np.cos(theta), np.sin(-theta), 0],
                 [np.sin(theta), np.cos(theta), 0],
                 [0, 0, 1]])
    T2 = np.array([[1, 0, -c_x],
                  [0, 1, -c_y],
                  [0, 0, 1]])
    M = T1 @ R @ T2
     
    for i in range(0, ((n-2)*4)+1, 4):
        r0[i+4] = M @ r0[i]
        r0[i+5] = M @ r0[i+1]
        r0[i+6] = M @ r0[i+2]
        r0[i+7] = M @ r0[i+3]
    return r0

In [13]:
import math

vcirculo = 64 # define a "qualidade" do circulo
vretangulos = 8 + (12 + 8 + 8 + 6)
vtriangulo = 3
vlinhas = 4

vtotal = (12*vcirculo) + vtriangulo + vretangulos + vlinhas + 50

vertices = np.zeros(vtotal, [("position", np.float32, 2)])

#############################################################################################################
#1: MACACO HIDRÁULICO - BOMBA + CABO + BALÃO + TRIÂNGULO

#círculo I:
pi = 3.1415
counter = 0
radius = 0.3
offset_x = +0.0
offset_y = -0.6

angle = 0.0
for counter in range(vcirculo):
    angle += 2*pi/vcirculo 
    x = math.cos(angle)*radius
    y = math.sin(angle)*radius
    vertices[counter] = [x+offset_x,y+offset_y]
    
print(f"vértices do círculo I: 0 a {counter}")
formercounter = counter
    
#triângulo II:
vertices[counter+1] = [ -0.15,-0.5]
vertices[counter+2] = [+0.0, -0.2]
vertices[counter+3] = [0.15, -0.5]
counter += 3

print(f"vértices do triângulo II: {formercounter} a {counter}")
formercounter = counter

#linha III
vertices[counter+1] = [+0.0, -0.6]
vertices[counter+2] = [-0.6, -0.6]
counter += 2

print(f"vértices da linha III: {formercounter} a {counter}")
formercounter = counter

#retângulo IV
vertices[counter+1] = [-0.9, -0.3] #G
vertices[counter+3] = [-0.6,-0.3] #J
vertices[counter+2] = [-0.9, -0.9] #H
vertices[counter+4] = [-0.6,-0.9] #I
counter += 4

print(f"vértices do retângulo IV: {formercounter} a {counter}")
formercounter = counter

#retângulo V
vertices[counter+1] = [-0.8, +0.0] #C
vertices[counter+2] = [-0.7, +0.0] #D
vertices[counter+3] = [-0.8,-0.3] #E
vertices[counter+4] = [-0.7,-0.3] #F
counter += 4

print(f"vértices do retângulo V: {formercounter} a {counter}")
formercounter = counter

#linha VI
vertices[counter+1] = [-0.9, +0.0]
vertices[counter+2] = [-0.6, +0.0]
counter += 2

print(f"vértices da linha VI: {formercounter} a {counter}")
formercounter = counter
#############################################################################################################
#2: ENGRENAGENS - CÍRCULOS + RETÂNGULOS

#círculo 1:
radius = 0.22
offset_x = +0.4
offset_y = +0

angle = 0.0
for i in range(counter+1, counter+vcirculo+1):
    angle += 2*pi/vcirculo 
    x = math.cos(angle)*radius
    y = math.sin(angle)*radius
    vertices[i] = [x+offset_x,y+offset_y]
    
counter += vcirculo

#-------------------------------------------------------------
#círculo 1.1:
radius = 0.02
offset_x = +0.45
offset_y = +0.05

angle = 0.0
for i in range(counter+1, counter+vcirculo+1):
    angle += 2*pi/vcirculo 
    x = math.cos(angle)*radius
    y = math.sin(angle)*radius
    vertices[i] = [x+offset_x,y+offset_y]
    
counter += vcirculo

#círculo 1.2:
radius = 0.02
offset_x = +0.45
offset_y = +-0.05

angle = 0.0
for i in range(counter+1, counter+vcirculo+1):
    angle += 2*pi/vcirculo 
    x = math.cos(angle)*radius
    y = math.sin(angle)*radius
    vertices[i] = [x+offset_x,y+offset_y]
    
counter += vcirculo

#círculo 1.3:
radius = 0.02
offset_x = +0.35
offset_y = +0.05

angle = 0.0
for i in range(counter+1, counter+vcirculo+1):
    angle += 2*pi/vcirculo 
    x = math.cos(angle)*radius
    y = math.sin(angle)*radius
    vertices[i] = [x+offset_x,y+offset_y]
    
counter += vcirculo

#círculo 1.4:
radius = 0.02
offset_x = +0.35
offset_y = -0.05

angle = 0.0
for i in range(counter+1, counter+vcirculo+1):
    angle += 2*pi/vcirculo 
    x = math.cos(angle)*radius
    y = math.sin(angle)*radius
    vertices[i] = [x+offset_x,y+offset_y]
    
counter += vcirculo

#--------------------------------------------------------------

# retângulos da primeira engrenagem
radius = 0.22
offset_x = +0.4
offset_y = +0
l = 0.08
h = 0.09
n1 = 12

r1 = calcula_retangulos(offset_x, offset_y, radius-0.045, l, h, n1)
counter += 1
for i in range(0, n1*4, 4):
    vertices[counter+i] = r1[i][:-1].tolist()
    vertices[counter+i+1] = r1[i+1][:-1].tolist()
    vertices[counter+i+2] = r1[i+2][:-1].tolist()
    vertices[counter+i+3] = r1[i+3][:-1].tolist()
    
counter += n1*4

#círculo 2:
radius = 0.15
offset_x = +0.65
offset_y = +0.4

angle = 0.0
for i in range(counter, counter+vcirculo):
    angle += 2*pi/vcirculo 
    x = math.cos(angle)*radius
    y = math.sin(angle)*radius
    vertices[i] = [x+offset_x,y+offset_y]
    
counter += vcirculo

#círculo 2.1:
radius = 0.03
offset_x = +0.7
offset_y = +0.45

angle = 0.0
for i in range(counter, counter+vcirculo):
    angle += 2*pi/vcirculo 
    x = math.cos(angle)*radius
    y = math.sin(angle)*radius
    vertices[i] = [x+offset_x,y+offset_y]
    
counter += vcirculo

#círculo 2.2:
radius = 0.03
offset_x = +0.6
offset_y = +0.35

angle = 0.0
for i in range(counter, counter+vcirculo):
    angle += 2*pi/vcirculo 
    x = math.cos(angle)*radius
    y = math.sin(angle)*radius
    vertices[i] = [x+offset_x,y+offset_y]
    
counter += vcirculo

#retângulos da segunda engrenagem
radius = 0.15
offset_x = +0.65
offset_y = +0.4

l = 0.06
h = 0.082
n2 = 8

r2 = calcula_retangulos(offset_x, offset_y, radius-0.04, l, h, n2)
#counter += 1
for i in range(0, n2*4, 4):
    vertices[counter+i] = r2[i][:-1].tolist()
    vertices[counter+i+1] = r2[i+1][:-1].tolist()
    vertices[counter+i+2] = r2[i+2][:-1].tolist()
    vertices[counter+i+3] = r2[i+3][:-1].tolist()
    
counter += n2*4

print("antes de c3", counter)

#círculo 3:
radius = 0.075
offset_x = +0.13
offset_y = +0.25

angle = 0.0
for i in range(counter, counter+vcirculo):
    angle += 2*pi/vcirculo 
    x = math.cos(angle)*radius
    y = math.sin(angle)*radius
    vertices[i] = [x+offset_x,y+offset_y]
    
counter += vcirculo

print("depois de c3", counter)

#círculo 3.1:
radius = 0.04
offset_x = +0.13
offset_y = +0.25

angle = 0.0
for i in range(counter, counter+vcirculo):
    angle += 2*pi/vcirculo 
    x = math.cos(angle)*radius
    y = math.sin(angle)*radius
    vertices[i] = [x+offset_x,y+offset_y]
    
counter += vcirculo

print("depois de c3.1", counter)

#retângulos da terceira engrenagem
radius = 0.075
offset_x = +0.13
offset_y = +0.25

l = 0.04
h = 0.035
n3 = 6

r3 = calcula_retangulos(offset_x, offset_y, radius-0.005, l, h, n3)
#counter += 1
for i in range(0, n3*4, 4):
    vertices[counter+i] = r3[i][:-1].tolist()
    vertices[counter+i+1] = r3[i+1][:-1].tolist()
    vertices[counter+i+2] = r3[i+2][:-1].tolist()
    vertices[counter+i+3] = r3[i+3][:-1].tolist()
    
counter += n3*4
print(counter)


vértices do círculo I: 0 a 63
vértices do triângulo II: 63 a 66
vértices da linha III: 66 a 68
vértices do retângulo IV: 68 a 72
vértices do retângulo V: 72 a 76
vértices da linha VI: 76 a 78
antes de c3 671
depois de c3 735
depois de c3.1 799
823


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot.

In [14]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)


### Abaixo, nós enviamos todo o conteúdo da variável vertices.

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [15]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associando variáveis do programa GLSL (Vertex Shaders) com nossos dados

Primeiro, definimos o byte inicial e o offset dos dados.

In [16]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Nós definimos essa variável no Vertex Shader.

In [17]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, nós indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

Outros parâmetros:

* Definimos que possui duas coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [18]:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

In [19]:
loc_color = glGetUniformLocation(program, "color")
R = 1.0
G = 0.0
B = 0.0

In [20]:
## MOVIMENTOS

#############################################################################################################

#MACACO HIDRÁULICO

#variáveis de translação
#desce a válvula
tv_y = 0

#sobe o triângulo
tt_y = 0

#infla o círculo
tc_y = 0
tc_x = 0
ec_x = 1
ec_y = 1

# rotaciona as engrenagens
theta = 0


def key_event(window,key,scancode,action,mods):
    global tv_y, tt_y, ec_x, ec_y, tc_y, theta
    
    print('[key event] key=',key)
    print('[key event] scancode=',scancode)
    print('[key event] action=',action)
    print('[key event] mods=',mods)
    print('-------')

    #desce a válvula:
    if key == 83 and tv_y > -0.1:
        #válvula e barra descem
        tv_y -= 0.0005
        #bola infla
        #antes chamaremos uma translação para centralizar o círculo
        tc_y = -0.6
        tc_x = 0
        ec_x *= 1.001
        ec_y *= 1.001
        #triângulo sobe
        tt_y += 0.001
    
    #sobe válvula:
    if key == 87 and tv_y < 0.000:
        #válvula e barra descem
        tv_y += 0.0005
        #bola desinfla
        ec_x /= 1.001
        ec_y /= 1.001
        #triângulo desce
        tt_y -= 0.001
        
    if key == 262:
        theta += 0.1
        
    if key == 263:
        theta -= 0.1

glfw.set_key_callback(window,key_event)
#############################################################################################################



<function __main__.key_event(window, key, scancode, action, mods)>

### Nesse momento, nós exibimos a janela!


In [21]:
glfw.show_window(window)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

In [22]:
while not glfw.window_should_close(window):

    glfw.poll_events() 

#############################################################################################################
#1: MACACO HIDRÁULICO - BOMBA + CABO + BALÃO + TRIÂNGULO

    # vértices do círculo I: 0 a 63
    # vértices do triângulo II: 63 a 66
    # vértices da linha III: 66 a 68
    # vértices do retângulo IV: 68 a 72
    # vértices do retângulo V: 72 a 76
    # vértices da linha VI: 76 a 78
    
    glClear(GL_COLOR_BUFFER_BIT) 
    glClearColor(0.5, 0.0, 0.5, 0.0)
    
                                                            #estáticos
                                                      
    mat_transf =  np.array([    1.0, 0.0, 0.0, 0.0, 
                                0.0, 1.0, 0.0, 0.0, 
                                0.0, 0.0, 1.0, 0.0, 
                                0.0, 0.0, 0.0, 1.0], np.float32)
    
    loc = glGetUniformLocation(program, "mat_transf")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transf)
    
    #linha III
    glUniform4f(loc_color, 1.0, 1.0, 1.0, 1.0)
    glDrawArrays(GL_LINES, 67, 2)
    
    #retângulo IV
    glUniform4f(loc_color, 0.2, 0.0, 0.0, 0.2)
    glDrawArrays(GL_TRIANGLE_STRIP, 69, 4)
    
                                                            #infla bola
    mat_transf =  np.array([    ec_x, 0.0, 0.0, (tc_x - tc_x*ec_x), 
                                0.0, ec_y, 0.0, (tc_y - tc_y*ec_y), 
                                0.0, 0.0, 1.0, 0.0, 
                                0.0, 0.0, 0.0, 1.0], np.float32)
    
    loc = glGetUniformLocation(program, "mat_transf")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transf)
    
    #circulo I
    glUniform4f(loc_color, 1.0, 0.0, 0.0, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, 0, 64)
    
                                                            #sobe triângulo
    mat_transf  =    np.array([    1.0, 0.0, 0.0, 0, 
                                   0.0, 1.0, 0.0, tt_y, 
                                   0.0, 0.0, 1.0, 0.0, 
                                   0.0, 0.0, 0.0, 1.0], np.float32)
    
    loc = glGetUniformLocation(program, "mat_transf")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transf)
    
    #triangulo II
    glUniform4f(loc_color, 0.3, 0.0, 1.0, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, 64, 3)
    
                                                            #desce válvula
    
    mat_transf  =    np.array([    1.0, 0.0, 0.0, 0, 
                                   0.0, 1.0, 0.0, tv_y, 
                                   0.0, 0.0, 1.0, 0.0, 
                                   0.0, 0.0, 0.0, 1.0], np.float32)
    print(mat_transf)
    
    loc = glGetUniformLocation(program, "mat_transf")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transf)
    
    #retângulo V
    glUniform4f(loc_color, 0.2, 0.0, 0.0, 0.2)
    glDrawArrays(GL_TRIANGLE_STRIP, 73, 4)
    
    #linha VI
    glUniform4f(loc_color, 1.0, 1.0, 1.0, 1.0)
    glDrawArrays(GL_LINES, 77, 2)
    
#############################################################################################################

    #ENGRENAGENS
    
    # matriz de transformação da 1a engrenagem
    c_x = +0.4
    c_y = 0.0
    T1 = np.array([[1, 0, 0, c_x],
                  [0, 1, 0, c_y],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]])
    R = np.array([[np.cos(theta), np.sin(-theta), 0, 0],
                 [np.sin(theta), np.cos(theta), 0, 0], 
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])
    T2 = np.array([[1, 0, 0, -c_x],
                  [0, 1, 0, -c_y],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]])
    M = T1 @ R @ T2
    mat_transf = M.flatten()
    
    loc = glGetUniformLocation(program, "mat_transf")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transf)
    
    # círculo 1   
    glUniform4f(loc_color, 1.0, 0.5, 0.0, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, 79, 64)
    
    # subcírculos 1 a 4
    glUniform4f(loc_color, 0.5, 0, 0.5, 1.0)
    for i in range(4):
        glDrawArrays(GL_TRIANGLE_FAN, 79+64+(64*i), 64)
    
    #79 + 64*5 = 399
    
    # retângulos da engrenagem 1
    glUniform4f(loc_color, 1.0, 0.5, 0.0, 1.0)
    n1 = 12
    for i in range(n1):
        glDrawArrays(GL_TRIANGLE_STRIP, 399+(i*4), 4)
        
    # 399 + 12*4 = 447
        
    # matriz de transformação da 2a engrenagem
    c_x = +0.65
    c_y = +0.4
    T1 = np.array([[1, 0, 0, c_x],
                  [0, 1, 0, c_y],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]])
    R = np.array([[np.cos(theta), np.sin(theta), 0, 0],
                 [np.sin(-theta), np.cos(theta), 0, 0], 
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])
    T2 = np.array([[1, 0, 0, -c_x],
                  [0, 1, 0, -c_y],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]])
    M = T1 @ R @ T2
    mat_transf = M.flatten()
    
    loc = glGetUniformLocation(program, "mat_transf")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transf)
    
    # círculo 2
    glUniform4f(loc_color, 1.0, 0.5, 0.0, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, 447, 64)
    
    # subcírculos 1 e 2
    glUniform4f(loc_color, 0.5, 0, 0.5, 1.0)
    
    glDrawArrays(GL_TRIANGLE_FAN, 447+64, 64)
    glDrawArrays(GL_TRIANGLE_FAN, 447+128, 64)
    
    # 447 + 3*64 = 639
    
    # retângulos da engrenagem 2    
    glUniform4f(loc_color, 1.0, 0.5, 0.0, 1.0)
    n2 = 8
    for i in range(n2):
        glDrawArrays(GL_TRIANGLE_STRIP, 639+(i*4), 4)
        
    # terceira engrenagem
    
    # matriz de transformação da 3a engrenagem
    c_x = +0.13
    c_y = +0.25
    T1 = np.array([[1, 0, 0, c_x],
                  [0, 1, 0, c_y],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]])
    R = np.array([[np.cos(theta), np.sin(theta), 0, 0],
                 [np.sin(-theta), np.cos(theta), 0, 0], 
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])
    T2 = np.array([[1, 0, 0, -c_x],
                  [0, 1, 0, -c_y],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]])
    M = T1 @ R @ T2
    mat_transf = M.flatten()
    
    loc = glGetUniformLocation(program, "mat_transf")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transf)
    
    # círculo 3
    glUniform4f(loc_color, 1.0, 0.5, 0.0, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, 671, 64)
    
    # subcírculo
    glUniform4f(loc_color, 0.5, 0, 0.5, 1.0)
    
    glDrawArrays(GL_TRIANGLE_FAN, 671+64, 64)
    
    # 671 + 128 = 799
    
    # retângulos da engrenagem 3   
    glUniform4f(loc_color, 1.0, 0.5, 0.0, 1.0)
    n3 = 6
    for i in range(n3):
        glDrawArrays(GL_TRIANGLE_STRIP, 799+(i*4), 4)
    
    glfw.swap_buffers(window)

glfw.terminate()

[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]


[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] key= 262
[key event] scancode= 114
[key event] action= 2
[key event] mods= 0
-------
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] key= 262
[key event] scancode= 114
[key event] action= 2
[key event] mods= 0
-------
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] key= 262
[key event] scancode= 114
[key event] action= 2
[key event] mods= 0
-------
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] key= 262
[key event] scancode= 114
[key event] action= 2
[key event] mods= 0
-------
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] 

[key event] key= 262
[key event] scancode= 114
[key event] action= 2
[key event] mods= 0
-------
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] key= 262
[key event] scancode= 114
[key event] action= 2
[key event] mods= 0
-------
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] key= 262
[key event] scancode= 114
[key event] action= 2
[key event] mods= 0
-------
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] key= 262
[key event] scancode= 114
[key event] action= 2
[key event] mods= 0
-------
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] key= 262
[key event] scancode= 114
[key event] action= 2
[key event] mods= 0
-------
[1. 0. 0. 0. 0.

[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]


[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] key= 262
[key event] scancode= 114
[key event] action= 2
[key event] mods= 0
-------
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] key= 262
[key event] scancode= 114
[key event] action= 2
[key event] mods= 0
-------
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] key= 262
[key event] scancode= 114
[key event] action= 2
[key event] mods= 0
-------
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] key= 262
[key event] scancode= 114
[key event] action= 2
[key event] mods= 0
-------
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] key= 262
[key event] scancode= 114
[key event] act

...

[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] key= 263
[key event] scancode= 113
[key event] action= 0
[key event] mods= 0
-------
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] key= 263
[key event] scancode= 113
[key event] action= 1
[key event] mods= 0
-------
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] key= 263
[key event] scancode= 113
[key even

[key event] key= 263
[key event] scancode= 113
[key event] action= 0
[key event] mods= 0
-------
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] key= 263
[key event] scancode= 113
[key event] action= 1
[key event] mods= 0
-------
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] key= 263
[key event] scancode= 113
[key event] action= 0
[key event] mods= 0
-------
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] key= 263
[key event] scancode= 113
[key event] action= 1
[key event] mods= 0
-------
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0.

[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] key= 263
[key event] scancode= 113
[key event] action= 0
[key event] mods= 0
-------
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] key= 263
[key event] scancode= 113
[key event] action= 1
[key event] mods= 0
-------
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key event] key= 263
[key event] scancode= 113
[key event] action= 0
[key event] mods= 0
-------
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[key even

[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]


[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
